In [1]:
# some imports
import os
from skimage.draw import polygon
from scipy import interpolate
import numpy as np
np.random.seed(seed=1)
from glob import glob
from matplotlib import pyplot as plt
%matplotlib notebook
from natsort import natsorted
import keras
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator

# import custom functions and viewing tools
from VisTools import multi_slice_viewer0, mask_viewer0
from KerasModel import BlockModel, dice_coef_loss

Using TensorFlow backend.


In [2]:
#~# some parameters to set for training #~#
# path to save best model weights
model_version = 1
model_weights_path = os.path.join(os.getcwd(),
                                  'BestModelWeights_v{:02d}.h5'.format(model_version))
# set number of subjects to be used for testing
test_num = 10
# set number of subjects to to be used for validation
val_num = 10
# whether to use data augmentation or not
augment = True
# how many iterations of data to train on
numEp = 2
# augmentation factor
augFact = 8

# set data directory
data_dir = os.path.join('/home','johnsonj118','output')
# find number of subjects
numSubjs = len(glob(os.path.join(data_dir, "input*.npy")))

In [3]:
# Randaomly select test subject indices
# numpy is seeded so this is repeatable
tv_inds = np.random.choice(numSubjs,test_num+val_num,replace=False)
test_inds = tv_inds[:test_num]
val_inds = tv_inds[test_num:]

In [4]:
input_files= natsorted(glob(os.path.join(data_dir, "input*.npy")))
print('{} input files found'.format(len(input_files)))
# split into test,train,validation
input_files_test = np.take(input_files,test_inds)
input_files_val = np.take(input_files,val_inds)
input_files_train = np.delete(input_files,tv_inds)

target_files= natsorted(glob(os.path.join(data_dir, "target*.npy")))
print('{} output files found'.format(len(target_files)))
# split into test and test/validation
target_files_test = np.take(target_files,test_inds)
target_files_val = np.take(target_files,val_inds)
target_files_train = np.delete(target_files,tv_inds)

44 input files found
44 output files found


In [5]:
# load input data
print('Loading input data...')
inputs_test = np.concatenate([np.load(f) for f in input_files_test])
inputs_val = np.concatenate([np.load(f) for f in input_files_val])
inputs_train = np.concatenate([np.load(f) for f in input_files_test])
# add singleton dimension for grayscale channel
testX = inputs_test[...,np.newaxis]
valX = inputs_val[...,np.newaxis]
trainX = inputs_train[...,np.newaxis]
print('Input data loaded')

Loading input data...
Input data loaded


In [6]:
# load input data
print('Loading target data...')
targets_test = np.concatenate([np.load(f) for f in target_files_test])
targets_val = np.concatenate([np.load(f) for f in target_files_val])
targets_train = np.concatenate([np.load(f) for f in target_files_test])
# add singleton dimension for grayscale channel
testY = targets_test[...,np.newaxis]
valY = targets_val[...,np.newaxis]
trainY = targets_train[...,np.newaxis]
print('Target data loaded')

Loading target data...
Target data loaded


In [8]:
# make model
model = BlockModel(trainX.shape,filt_num=16,numBlocks=4)
model.compile(optimizer=Adam(), loss=dice_coef_loss)

In [10]:
# setup image data generator
if augment:
    datagen1 = ImageDataGenerator(
        rotation_range=15,
        shear_range=0.5,
        width_shift_range=0.1,
        height_shift_range=0.1,
        zoom_range=0.2,
        horizontal_flip=True,
        vertical_flip=True,
        fill_mode='nearest')
    datagen2 = ImageDataGenerator(
        rotation_range=15,
        shear_range=0.5,
        width_shift_range=0.1,
        height_shift_range=0.1,
        zoom_range=0.2,
        horizontal_flip=True,
        vertical_flip=True,
        fill_mode='nearest')
else:
    datagen1 = ImageDataGenerator()
    datagen2 = ImageDataGenerator()
# Provide the same seed and keyword arguments to the fit and flow methods
seed = 1
datagen1.fit(trainX, seed=seed)
datagen2.fit(trainY, seed=seed)
batchsize = 16
datagen = zip( datagen1.flow( trainX, None, batchsize, seed=seed), datagen2.flow( trainY, None, batchsize, seed=seed) )

# calculate number of batches
if augment:
    steps = np.int(trainX.shape[0]/batchsize*augFact)
else:
    steps = np.int(trainX.shape[0]/batchsize)

In [16]:
# make callback for checkpointing
cb_check = ModelCheckpoint(model_weights_path,monitor='val_loss',
                                   verbose=0,save_best_only=True,
                                   save_weights_only=True,mode='auto',period=1)

In [ ]:
# train model
model.fit_generator(datagen,
                    steps_per_epoch=steps,
                    epochs=numEp,
                    callbacks=[cb_check],
                    verbose=1,
                    validation_data=(valX,valY))

Epoch 1/2


In [ ]:
# load best weights
model.load_weights(model_weights_path)
# evaluate on test data
score = model.evaluate(testX,testY,verbose=0)
print("Test Dice score is {:.03f}".format(1-score))

# display some results
output = model.predict(testX,batch_size=16)
mask_viewer0(testX[...,0],testY[...,0],output[...,0],name='Test Results')
plt.show()